<img src="logo.png" align="right" style="float" width="400">
<font color="#EF4123"><h1 align="right">Dude, where's my shared car?</h1></font>

**Jupyter 02 - Clima de Madrid**
--------------------------------
-----------------------------------------------------------------------
Cargamos datos del tiempo a nivel local en periodos de una semana o en las próximas 24 horas desde la web de AEMET OpenData. 

Previamente a cualquier consulta, se requiere solicitar una api-key personal a través del siguiente enlace: https://opendata.aemet.es/centrodedescargas/altaUsuario?. Por otra parte, los métodos de consulta están documentados en https://opendata.aemet.es/dist/index.html?.

Datos del Tiempo
---------------
Los métodos de consulta del API REST de AEMET se han implementado en el fichero 'functions_aemet.py'

In [1]:
import pandas as pd

import urllib.request, json 

from functions_aemet import consulta_historico_todas_aemet
from functions_aemet import consulta_historico_estacion_aemet
from functions_aemet import consulta_diaria_aemet
from functions_aemet import consulta_horaria_aemet
from functions_aemet import consulta_historico_todas_aemet
from functions_aemet import consulta_historico_estacion_aemet
from functions_aemet import consulta_historico_anno_estacion_aemet

Configuraciones previas

In [2]:
plotear_en_notebook = True # en caso de False, en un html externo

Carga del api-key, que está almacenada en un CSV externo

In [3]:
# read api-key
apy_key = pd.read_csv('Datos/api-key.csv')
apikey = pd.DataFrame(apy_key, columns=['site','key']).ix[0,'key']
#print(apykey)

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


Carga de predicciones climáticas para las próximas horas

In [4]:
# configuraciones previas
localidad = "26036" #"26036", Calahorra; "28079", Madrid 
periodo = "horaria"  #"horaria"; "diaria", 1 semana
query_semana = ['uvMax','fecha','humedadRelativa.maxima','humedadRelativa.minima','sensTermica.minima','sensTermica.maxima','temperatura.maxima','temperatura.minima']
API = "prediccion/especifica/municipio"

In [5]:
datos_aemet_semana, fecha_consulta = consulta_diaria_aemet(apikey, localidad, query_semana, API)
#print(datos_aemet_semana)
#print(fecha_consulta)

prediccion_hoy_1h, prediccion_hoy_6h = consulta_horaria_aemet(apikey, localidad, API)
#print(prediccion_hoy_1h)
#print(prediccion_hoy_6h)

Carga del histórico (máximo de un mes)

In [6]:
API = "valores/climatologicos/diarios/datos"

#fechaini = fecha_consulta + "T00%3A00%3A00UTC"
fechaini = "2018-10-01T01%3A01%3A01UTC"
fechafin = "2018-11-01T01%3A01%3A01UTC"

#data = consulta_historico_todas_aemet(apikey, API, fechaini, fechafin)
#print(data)

In [7]:
API = "valores/climatologicos/diarios/datos"

# fechas
fechaini = "2018-10-01T01%3A01%3A01UTC"
#fechaini = fecha_consulta + "T00%3A00%3A00UTC"
fechafin = "2018-11-01T01%3A01%3A01UTC"

# estaciones
ciudad_universitaria = "3194U" 
aeropuerto = "3129" 
retiro = "3195" 
cuatro_vientos = "3196" 

historico_cu = consulta_historico_estacion_aemet(apikey, ciudad_universitaria, API, fechaini, fechafin)
#print(historico_cu)
historico_aero = consulta_historico_estacion_aemet(apikey, aeropuerto, API, fechaini, fechafin)
#print(historico_aero)
historico_retiro = consulta_historico_estacion_aemet(apikey, retiro, API, fechaini, fechafin)
#print(historico_retiro)
historico_4vientos = consulta_historico_estacion_aemet(apikey, cuatro_vientos, API, fechaini, fechafin)
#print(historico_4vientos)

Almacenamos el histórico de una estaciónde un año

In [8]:
anno = "2010"
df = consulta_historico_anno_estacion_aemet(apikey, API,  retiro, anno)
columns = ['dir','fecha','horaPresMax','horaPresMin','horaracha','horatmax','horatmin','nombre','prec','presMax','presMin','racha','tmax','tmed','tmin','velmedia']
df = df[columns]
#print(df)

# almacenar fichero
filename = "Datos/climatologia_madrid_" + anno + ".csv"
df.to_csv(filename, sep=',', encoding='utf-8', index=False)

C:\Users\Miguel\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Almacenamos el histórico de una estación durante varios años

In [9]:
annos = ["2010","2011","2012","2013","2014","2015","2016","2017","2018"]

df = pd.DataFrame()
for anno in annos:
    print(anno)
    df_aux = consulta_historico_anno_estacion_aemet(apikey, API,  retiro, anno)
    #columns = ['dir','fecha','horaPresMax','horaPresMin','horaracha','horatmax','horatmin','nombre','prec','presMax','presMin','racha','tmax','tmed','tmin','velmedia']
    #df_aux = df_aux[columns]
    df = df.append(df_aux, ignore_index=True)

# filtramos columnas
columns = ['dir','fecha','horaPresMax','horaPresMin','horaracha','horatmax','horatmin','nombre','prec','presMax','presMin','racha','tmax','tmed','tmin','velmedia']
df = df[columns]
    
# almacenar fichero
filename = "Datos/climatologia_madrid.csv"
df.to_csv(filename, sep=',', encoding='utf-8', index=False)

2010


C:\Users\Miguel\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


2011
2012
2013
2014
2015
2016
2017
2018


Bokeh
------

Usaremos el paquete *Bokeh* para tener representaciones interactivas

In [10]:
from bokeh.palettes import viridis,inferno,grey, cividis, plasma, magma
from bokeh.io import output_notebook
from bokeh.models import GeoJSONDataSource, HoverTool, CategoricalColorMapper, LinearColorMapper
from bokeh.plotting import figure, show
from bokeh.io import output_file, show
from bokeh.models import ColorBar, LinearAxis, Range1d, ColumnDataSource
from bokeh.models.widgets import CheckboxGroup
from bokeh.models.ranges import Range1d
from bokeh.layouts import row

In [15]:
ds = ColumnDataSource(prediccion_hoy_1h)
#print(ds.data)

# create a plot and style its properties
prediccion_hoy_1 = figure(plot_width=500, plot_height=500)
prediccion_hoy_1.title.text = "Madrid: predicción para el " + fecha_consulta
prediccion_hoy_1.x_range = Range1d(7, 23, bounds=(0, None))
prediccion_hoy_1.y_range = Range1d(0, 20, bounds=(0, None))
prediccion_hoy_1.xaxis.axis_label = 'Hora'
prediccion_hoy_1.yaxis.axis_label = 'Temperatura'
glyph_1 = prediccion_hoy_1.line('periodo','temperatura',source=ds, legend='Temperatura', color='#dc3700', line_width=3, alpha = 0.8)
glyph_2 = prediccion_hoy_1.line('periodo','viento',source=ds, legend='Viento', color='#a5dc00', line_width=3, alpha = 0.8, y_range_name="extra_y")
prediccion_hoy_1.add_tools(HoverTool(renderers=[glyph_1], tooltips=[("Hora","@periodo"),("Temperatura","@temperatura")],mode='vline'))
prediccion_hoy_1.add_tools(HoverTool(renderers=[glyph_2], tooltips=[("Hora","@periodo"),("Viento","@viento")],mode='vline'))
prediccion_hoy_1.extra_y_ranges = {"extra_y": Range1d(start=0, end=50)}
prediccion_hoy_1.add_layout(LinearAxis(y_range_name="extra_y", axis_label='Viento'), 'right')

prediccion_hoy_2 = figure(plot_width=500, plot_height=500)
prediccion_hoy_2.title.text = "Madrid: predicción para el " + fecha_consulta
prediccion_hoy_2.x_range = Range1d(7, 23, bounds=(0, None))
prediccion_hoy_2.y_range = Range1d(0, 20, bounds=(0, None))
prediccion_hoy_2.xaxis.axis_label = 'Hora'
prediccion_hoy_2.yaxis.axis_label = 'Precipitacion'
#prediccion_hoy_2.add_layout(LinearAxis(y_range_name="foo2", axis_label='Humedad'), 'right')
glyph_3 = prediccion_hoy_2.line('periodo','precipitacion',source=ds, legend='Precipitacion', color='#00a5dc', line_width=3, alpha = 0.8)
glyph_4 = prediccion_hoy_2.line('periodo','humedad_relativa',source=ds, legend='Humedad', color='#00dc37', line_width=3, alpha = 0.8, y_range_name="extra_y")
prediccion_hoy_2.add_tools(HoverTool(renderers=[glyph_3], tooltips=[("Hora","@periodo"),("Precipitación","@precipitacion")],mode='vline'))
prediccion_hoy_2.add_tools(HoverTool(renderers=[glyph_4], tooltips=[("Hora","@periodo"),("Humedad relativa","@humedad_relativa")],mode='vline'))
prediccion_hoy_2.extra_y_ranges = {"extra_y": Range1d(start=0, end=100)}
prediccion_hoy_2.add_layout(LinearAxis(y_range_name="extra_y", axis_label='Humedad Relativa'), 'right')

# mostrar en notebook o en html
if (plotear_en_notebook):
    output_notebook()
else:
    output_file("prediccion_tiempo.html")
    
show(row(prediccion_hoy_1, prediccion_hoy_2))

Loading BokehJS ...